In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import LawSchoolDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning2

from source.experiment_interface import run_exp_iter_with_LFR

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'LFR_law_school'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [{'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': 50.0}]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp', 'LFR', EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                'LFR', EXPERIMENT_NAME, 'law_school_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'a1345526-1623-4fc7-b1a4-2a7790c4eef5',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  a1345526-1623-4fc7-b1a4-2a7790c4eef5


## Initialize custom objects

In [9]:
data_loader = LawSchoolDataset()
data_loader.X_data.head()

,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,male,tier,race
0,10.0,10.0,44.0,3.5,1.33,1.88,1.0,5.0,0.0,4.0,White
1,5.0,4.0,29.0,3.5,-0.11,-0.57,1.0,4.0,0.0,2.0,White
2,8.0,7.0,37.0,3.4,0.63,0.37,1.0,3.0,1.0,4.0,White
3,8.0,7.0,43.0,3.3,0.67,0.34,1.0,4.0,0.0,4.0,White
4,3.0,2.0,41.0,3.3,-0.67,-1.30,1.0,4.0,0.0,5.0,White


In [10]:
data_loader.X_data.shape

(20798, 11)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [12]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      with_tuning=True,
                      # with_tuning=False,
                      # tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='LawSchoolDataset',
                      verbose=True)

2024-01-02 19:45:56 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "[{'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': "
                                 '50.0}]',
 'model_init_seed': 100,
 'session_uuid': 'a1345526-1623-4fc7-b1a4-2a7790c4eef5'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': 50.0}
step: 0, loss: 0.9307546467902039, L_x: 1.2534922358168135,  L_y: 0.5875978401576478,  L_z: 0.00661243768548776
step: 250, loss: 0.8012737070029219, L_x: 1.253867261663526,  L_y: 0.5667908936593447,  L_z: 0.004438882814538842
step: 500, loss: 0.6507900098742686, L_x: 1.2548118271917443,  L_y: 0.5378100741586864,  L_z: 0.002008636348873294
step: 750, loss: 0.578930238985534, L_x: 1.2554923452991456,  L_y: 0.5276429820132806,  L_z: 0.0007746466703852373
step: 1000, loss: 0.5594710480020171, L_x: 1.2554658046607585,  L_y: 0.528332631922053,  L_z: 0.0003716751606671287
step: 1250, loss: 0.5569522842400385, L_x: 1.2555340547307068,  L_y: 0.5270312799587231,  L_z: 0.0003473132746801666
step: 1500, loss: 0.5908517424089845, L_x: 1.256384364054489,  L_y: 0.506856016026366,  L_z: 0.0014286376548414748
step: 1750, loss: 0.542491634823469, L_x: 1.2557191709475481,  L_y: 0.5224982876946436,  L_z: 0.00014872310838699997
step: 2000, 

2024-01-02 19:46:21 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__fulltime_1.0', 'cat__fulltime_2.0', 'cat__fam_inc_1.0',
       'cat__fam_inc_2.0', 'cat__fam_inc_3.0', 'cat__fam_inc_4.0',
       'cat__fam_inc_5.0', 'cat__tier_1.0', 'cat__tier_2.0', 'cat__tier_3.0',
       'cat__tier_4.0', 'cat__tier_5.0', 'cat__tier_6.0', 'num__decile1b',
       'num__decile3', 'num__lsat', 'num__ugpa', 'num__zfygpa', 'num__zgpa'],
      dtype='object')
Top indexes of an X_test in the current base flow dataset:  Int64Index([ 7102,   593, 18841,  5078, 14172,  8064, 13554, 13401, 17015,
            18446,  6938,  3450,  9375, 19994, 16100,  4401,   142, 15143,
             2188,  4332],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([ 7102,   593, 18841,  5078, 14172,  8064, 13554, 13401, 17015,
            18446,  6938,  3450,  9375, 19994, 16100,  4401,   142, 15143,
             2188,  4332],
           dtype='int64')
2024/01/02, 19:46:21: Tuning MLPClassifie

2024-01-02 19:47:35 experiment_interface.py INFO    : Models are tuned and saved to a file
INFO:root:Models are tuned and saved to a file


Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [21]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_20240103__004735.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [22]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='LawSchoolDataset',
                      verbose=True)

2024-01-02 14:09:14 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 200,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7989742365933232, L_x: 1.0333738605431104,  L_y: 0.6789815789838769,  L_z: 0.008327635777567639
step: 250, loss: 0.7989741666340583, L_x: 1.0333738969712114,  L_y: 0.6789815068392818,  L_z: 0.008327635048827662
step: 500, loss: 0.7989742485844722, L_x: 1.0333738780784012,  L_y: 0.6789815929472827,  L_z: 0.00832763391467471
step: 750, loss: 0.6391934258825521, L_x: 1.0332909502036012,  L_y: 0.5194029750388243,  L_z: 0.00823067791168386
step: 1000, loss: 0.6391934637366692, L_x: 1.033290960107876,  L_y: 0.5194030101634384,  L_z: 0.008230678781221638
step: 1250, loss: 0.5228079191602457, L_x: 1.0314852347504928,  L_y: 0.403770397229017,  L_z: 0.007944499228089695
step: 1500, loss: 0.5228079383442957, L_x: 1.0314852594486816,  L_y: 0.4037704005509528,  L_z: 0.007944505924237373
step: 1750, loss: 0.5192984513523966, L_x: 1.0302606206214335,  L_y: 0.4009645894024503,  L_z: 0.007653899943901492
step: 2000, loss

2024-01-02 14:09:17 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:09:17 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [23]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_20240103__004735.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [24]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='LawSchoolDataset',
                      verbose=True)

2024-01-02 14:11:34 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 300,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7979782676913761, L_x: 1.0323907659454599,  L_y: 0.6781985204253934,  L_z: 0.00827033533571835
step: 250, loss: 0.797978199748413, L_x: 1.0323908019191732,  L_y: 0.6781984500730722,  L_z: 0.008270334741711762
step: 500, loss: 0.7979782787217018, L_x: 1.0323907832154928,  L_y: 0.6781985335275081,  L_z: 0.00827033343632219
step: 750, loss: 0.6475774505477387, L_x: 1.0322746489837868,  L_y: 0.5280085073128342,  L_z: 0.008170739168262924
step: 1000, loss: 0.6475774873926531, L_x: 1.0322746589152256,  L_y: 0.5280085418657529,  L_z: 0.008170739817688817
step: 1250, loss: 0.5396801755981412, L_x: 1.0305529553397612,  L_y: 0.42116019193809506,  L_z: 0.007732344063034973
step: 1500, loss: 0.5396801895434374, L_x: 1.0305529807322427,  L_y: 0.42116019036751473,  L_z: 0.007732350551349179
step: 1750, loss: 0.5361632824016606, L_x: 1.0293364091548245,  L_y: 0.41834810283738577,  L_z: 0.007440769324396114
step: 2000, 

2024-01-02 14:11:37 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:11:37 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [25]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_20240103__004735.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [26]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='LawSchoolDataset',
                      verbose=True)

2024-01-02 14:13:52 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 400,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7990623753426728, L_x: 1.0328282366486268,  L_y: 0.6788628044478547,  L_z: 0.00845837361497765
step: 250, loss: 0.799062310279977, L_x: 1.0328282720631976,  L_y: 0.6788627364890443,  L_z: 0.008458373292306452
step: 500, loss: 0.7990623867186859, L_x: 1.0328282538498756,  L_y: 0.6788628177803322,  L_z: 0.008458371776683092
step: 750, loss: 0.653315875429133, L_x: 1.0327228893027587,  L_y: 0.5333254837317553,  L_z: 0.008359051383550958
step: 1000, loss: 0.6533159097569131, L_x: 1.032722899285175,  L_y: 0.5333255157988329,  L_z: 0.008359052014781295
step: 1250, loss: 0.5497125316449264, L_x: 1.031007016982998,  L_y: 0.4307364335163241,  L_z: 0.00793769821515125
step: 1500, loss: 0.5497125427775167, L_x: 1.031007042805463,  L_y: 0.43073643312089216,  L_z: 0.007937702688039144
step: 1750, loss: 0.5462092990230252, L_x: 1.029785484764563,  L_y: 0.4278213886820478,  L_z: 0.007704680932260533
step: 2000, loss: 0

2024-01-02 14:13:55 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:13:55 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [11]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_20240103__004735.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [12]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='LawSchoolDataset',
                      verbose=True)

2024-01-02 19:57:19 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': "[{'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': "
                                 '50.0}]',
 'model_init_seed': 500,
 'session_uuid': 'a1345526-1623-4fc7-b1a4-2a7790c4eef5'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': 50.0}
step: 0, loss: 0.9694259265878906, L_x: 1.2605184808625527,  L_y: 0.5879319675564437,  L_z: 0.007377775484456428
step: 250, loss: 0.7970774995364713, L_x: 1.2609658639957364,  L_y: 0.5673874733355443,  L_z: 0.0043416073512193955
step: 500, loss: 0.6710913783102783, L_x: 1.261894594093848,  L_y: 0.538910632454881,  L_z: 0.0023912359982891764
step: 750, loss: 0.5970292284432637, L_x: 1.262798217768101,  L_y: 0.5254937980882705,  L_z: 0.0011781489635462438
step: 1000, loss: 0.6170148520737906, L_x: 1.26323197385866,  L_y: 0.5213297547170035,  L_z: 0.0016610555523640103
step: 1250, loss: 0.5954041595440895, L_x: 1.262980599910945,  L_y: 0.5230708280301695,  L_z: 0.0011940705102962135
step: 1500, loss: 0.5824234903613081, L_x: 1.2630586839417564,  L_y: 0.5126776015990666,  L_z: 0.0011423060384564809
step: 1750, loss: 0.5572705984932246, L_x: 1.2630304620018449,  L_y: 0.5164445148074733,  L_z: 0.0005639155813146557
step: 2000

2024-01-02 19:57:45 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 19:57:45 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__fulltime_1.0', 'cat__fulltime_2.0', 'cat__fam_inc_1.0',
       'cat__fam_inc_2.0', 'cat__fam_inc_3.0', 'cat__fam_inc_4.0',
       'cat__fam_inc_5.0', 'cat__tier_1.0', 'cat__tier_2.0', 'cat__tier_3.0',
       'cat__tier_4.0', 'cat__tier_5.0', 'cat__tier_6.0', 'num__decile1b',
       'num__decile3', 'num__lsat', 'num__ugpa', 'num__zfygpa', 'num__zgpa'],
      dtype='object')
Top indexes of an X_test in the current base flow dataset:  Int64Index([12269, 18551, 11422,  4842,  2061, 12046, 16727, 11861, 14580,
            19701, 12917,  7538, 17610, 19120,  5581, 18072, 13209, 20500,
            14741, 11309],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([12269, 18551, 11422,  4842,  2061, 12046, 16727, 11861, 14580,
            19701, 12917,  7538, 17610, 19120,  5581, 18072, 13209, 20500,
            14741, 11309],
           dtype='int64')
Path for tuned params:  /home/dh3553/proj

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [29]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_20240103__004735.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [30]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='LawSchoolDataset',
                      verbose=True)

2024-01-02 14:18:23 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 600,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.8012308209526325, L_x: 1.032486691591748,  L_y: 0.6816260094522957,  L_z: 0.00817807117058102
step: 250, loss: 0.8012307592510307, L_x: 1.03248672607275,  L_y: 0.6816259452091471,  L_z: 0.00817807071730432
step: 500, loss: 0.8012308321514614, L_x: 1.032486709583512,  L_y: 0.6816260226759697,  L_z: 0.00817806925857021
step: 750, loss: 0.6601098419829545, L_x: 1.0323946558803412,  L_y: 0.5407171178798217,  L_z: 0.00807662925754936
step: 1000, loss: 0.66010987855932, L_x: 1.0323946661335703,  L_y: 0.5407171500610479,  L_z: 0.00807663094245753
step: 1250, loss: 0.5619558254545324, L_x: 1.0308029125968043,  L_y: 0.443589170867648,  L_z: 0.007643181663601961
step: 1500, loss: 0.5619558401906308, L_x: 1.0308029386365294,  L_y: 0.4435891757918725,  L_z: 0.007643185267552722
step: 1750, loss: 0.5585636756338709, L_x: 1.0296490241047005,  L_y: 0.44074622164810573,  L_z: 0.007426275787647554
step: 2000, loss: 0.558

2024-01-02 14:18:26 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:18:26 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]